In [1]:
import json
import pandas as pd
import numpy as np
import datetime

In [2]:
from os import listdir
from os.path import isfile, join
mypath='data'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [4]:
for d in onlyfiles:
    f = open('data/'+d)
    data = json.load(f)

In [418]:
team_info = pd.DataFrame({'team_name' : [], 'match_id' : [], 
                            'runs_scored' : [], 'wickets_lost' : [],
                            'runs_conceded' : [], 'wickets_taken' : [], 'date' : [], 'comp' : []})

player_info = pd.DataFrame({'player_name' : [], 'player_team' : [], 'match_id' : [],
                 'player_gender' : [], 'runs_scored' : [], 'runs_conceded' : [],
                 'wickets_taken' : [], 'out' : [], 'catches' : [], 'balls_faced' : [],
                 'fours' : [], 'sixes' : [], 'extras' : [], 'POM' : [], 'date' : []})
match_ids = []
h_teams = []
a_teams = []
cities = []
venues = []
dates = []
genders = []
toss_winners = []
toss_decisions = []
first_inn_runs = []
first_inn_wicks = []
second_inn_runs = []
second_inn_wicks = []
winners = []
outcomes = []
bat_firsts = []
fballs = []
sballs = []

sub_catches = {}
match_number = 1


for d in onlyfiles:
    f = open('data/'+d)
    data = json.load(f)
    
    # collect game info
    
    city = data['info']['city']
    cities.append(city)
    venue = data['info']['venue']
    venues.append(venue)
    date = data['info']['dates'][0]
    dates.append(date)

    season = data['info']['season']
    
    gender = data['info']['gender']
    genders.append(gender)
    
    
    match_ids.append(match_number)
    
    h_team = data['info']['teams'][1]
    h_teams.append(h_team)
    a_team = data['info']['teams'][0]
    a_teams.append(a_team)
    
    
    
    toss_win = data['info']['toss']['winner']
    toss_winners.append(toss_win)
    toss_outcome = data['info']['toss']['decision']
    toss_decisions.append(toss_outcome)
    
    try:
        pom = data['info']['player_of_match'][0]
        winner = data['info']['outcome']['winner']
        winners.append(winner)
        if 'wickets' in data['info']['outcome']['by'].keys():
            outcome = str(data['info']['outcome']['by']['wickets']) + ' wickets'
        else:
            outcome = str(data['info']['outcome']['by']['runs']) + ' runs'
        outcomes.append(outcome)
    except:
        winners.append(np.nan)
        outcomes.append('no result')
        
    
    
    if toss_outcome ==  'bat':
        bat_first = toss_win
    else:
        if toss_win == h_team:
            bat_first = a_team
        else:
            bat_first = h_team
    bat_firsts.append(bat_first)
    
    h_players = data['info']['players'][h_team]

    h_game_runs_scored = {}
    h_game_runs_con = {}
    h_game_wick = {}
    h_game_out = {}
    h_catches = {}
    h_balls = {}
    h_fours = {}
    h_sixes = {}
    h_extras = {}
    h_balls_bowled = {}

    for p in h_players:
        h_game_runs_scored[p] = 0
        h_game_runs_con[p] = 0
        h_game_wick[p] = 0
        h_game_out[p] = 0
        h_catches[p] = 0
        h_balls[p] = 0
        h_fours[p] = 0
        h_sixes[p] = 0
        h_extras[p] = 0
        h_balls_bowled[p] = 0

    a_players = data['info']['players'][a_team]

    a_game_runs_scored = {}
    a_game_runs_con = {}
    a_game_wick = {}
    a_game_out = {}
    a_catches = {}
    a_balls = {}
    a_fours = {}
    a_sixes = {}
    a_extras = {}
    a_balls_bowled = {}

    for p in a_players:
        a_game_runs_scored[p] = 0
        a_game_runs_con[p] = 0
        a_game_wick[p] = 0
        a_game_out[p] = 0
        a_catches[p] = 0
        a_balls[p] = 0
        a_fours[p] = 0
        a_sixes[p] = 0
        a_extras[p] = 0
        a_balls_bowled[p] = 0

    

    runs_scored = {a_team : a_game_runs_scored, h_team : h_game_runs_scored} 
    runs_con = {a_team : a_game_runs_con, h_team : h_game_runs_con} 
    wickets = {a_team : a_game_wick, h_team : h_game_wick} 
    outs = {a_team : a_game_out, h_team : h_game_out} 
    catches = {a_team: a_catches, h_team : h_catches}
    balls = {a_team : a_balls, h_team : h_balls}
    fours = {a_team : a_fours, h_team : h_fours}
    sixes = {a_team : a_sixes, h_team : h_sixes}
    extras = {a_team : a_extras, h_team : h_extras}
    balls_bowled = {a_team : a_balls_bowled, h_team : h_balls_bowled}
    
    # first_innings_collection
    try:
        fteam = data['innings'][0]['team']

        if fteam == h_team:
            fbowl_team = a_team
        else:
            fbowl_team = h_team

        fg_runs_scored = 0
        finn_wicks = 0
        fball_bowled = 0


        for o in data['innings'][0]['overs']:

            set_runs = 0
            set_wick = 0

            for d in o['deliveries']:
                bat = d['batter']
                bowl = d['bowler']
                
                run = d['runs']['batter']
                if run == 4:
                    fours[fteam][bat] += 1
                if run == 6:
                    sixes[fteam][bat] += 1
                runs_scored[fteam][bat] += run
                extra = d['runs']['extras']

                if 'extras' not in d.keys():
                    balls[fteam][bat] += 1
                    fball_bowled += 1
                    balls_bowled[fbowl_team][bowl] += 1
                else:
                    if 'byes' in d['extras'].keys() or 'legbyes' in d['extras'].keys() or 'noballs' in d['extras'].keys():
                        balls[fteam][bat] += 1
                        
                    if 'wides' in d['extras'].keys() or 'noballs' in d['extras'].keys():
                        extras[fbowl_team][bowl] += extra                   
                    if 'byes' in d['extras'].keys() or 'legbyes' in d['extras'].keys():
                        fball_bowled += 1
                        balls_bowled[fbowl_team][bowl] += 1
                        
                runs_con[fbowl_team][bowl] += run + extra

                set_runs += run
                set_runs += extra

                if 'wickets' in d.keys():
                    wickets[fbowl_team][bowl] += 1
                    outs[fteam][d['wickets'][0]['player_out']] += 1
                    if d['wickets'][0]['kind'] == 'caught':
                        if 'substitute' not in d['wickets'][0]['fielders'][0].keys():
                            catches[fbowl_team][d['wickets'][0]['fielders'][0]['name']] +=1
                        else:
                            if d['wickets'][0]['fielders'][0]['name'] not in sub_catches.keys():
                                sub_catches[d['wickets'][0]['fielders'][0]['name']] = 1
                            else:
                                sub_catches[d['wickets'][0]['fielders'][0]['name']] += 1
                    elif d['wickets'][0]['kind'] == 'caught and bowled':
                        catches[fbowl_team][bowl] += 1

                    set_wick += 1
            fg_runs_scored += set_runs
            finn_wicks += set_wick

        first_inn_runs.append(fg_runs_scored)
        first_inn_wicks.append(finn_wicks)
        fballs.append(fball_bowled)
    except:
        fg_runs_scored = np.nan
        fball_bowled = np.nan
        finn_wickets = np.nan
        first_inn_runs.append(np.nan)
        first_inn_wicks.append(np.nan)
        fballs.append(fball_bowled)
    
    # second_inn_data_collection
    try:
        steam = data['innings'][1]['team']
    
        if steam == h_team:
            sbowl_team = a_team
        else:
            sbowl_team = h_team

        sg_runs_scored = 0
        sinn_wicks = 0
        sball_bowled = 0

        for o in data['innings'][1]['overs']:

            set_runs = 0
            set_wick = 0

            for d in o['deliveries']:
                bat = d['batter']
                bowl = d['bowler']
                
                run = d['runs']['batter']
                if run == 4:
                    fours[steam][bat] += 1
                if run == 6:
                    sixes[steam][bat] += 1
                runs_scored[steam][bat] += run
                extra = d['runs']['extras']

                if 'extras' not in d.keys():
                    balls[steam][bat] += 1
                    sball_bowled += 1
                    balls_bowled[sbowl_team][bowl] += 1
                else:
                    if 'byes' in d['extras'].keys() or 'legbyes' in d['extras'].keys() or 'noballs' in d['extras'].keys():
                        balls[steam][bat] += 1
                        
                    if 'wides' in d['extras'].keys() or 'noballs' in d['extras'].keys():
                        extras[sbowl_team][bowl] += extra
                    if 'byes' in d['extras'].keys() or 'legbyes' in d['extras'].keys():
                        sball_bowled += 1
                        balls_bowled[sbowl_team][bowl] += 1


                runs_con[sbowl_team][bowl] += run + extra

                set_runs += run
                set_runs += extra

                if 'wickets' in d.keys():
                    wickets[sbowl_team][bowl] += 1
                    outs[steam][d['wickets'][0]['player_out']] += 1
                    if d['wickets'][0]['kind'] == 'caught':
                        if 'substitute' not in d['wickets'][0]['fielders'][0].keys():
                            catches[sbowl_team][d['wickets'][0]['fielders'][0]['name']] +=1
                        else:
                            if d['wickets'][0]['fielders'][0]['name'] not in sub_catches.keys():
                                sub_catches[d['wickets'][0]['fielders'][0]['name']] = 1
                            else:
                                sub_catches[d['wickets'][0]['fielders'][0]['name']] += 1
                    elif d['wickets'][0]['kind'] == 'caught and bowled':
                        catches[sbowl_team][bowl] += 1

                    set_wick += 1
            sg_runs_scored += set_runs
            sinn_wicks += set_wick
        second_inn_runs.append(sg_runs_scored)
        second_inn_wicks.append(sinn_wicks)
        sballs.append(sball_bowled)
    except:
        sg_runs_scored = np.nan
        sinn_wickets = np.nan
        sball_bowled = np.nan
        second_inn_runs.append(np.nan)
        second_inn_wicks.append(np.nan)
        sballs.append(sball_bowled)
        
    if h_team == fteam:
        rs = fg_runs_scored
        wl = finn_wicks
        rc = sg_runs_scored
        wt = sinn_wicks
        bb = fball_bowled
        bf = sball_bowled
        bat_first = True
        bat_second = False
        ht = True
        at = False
    else:
        rs = sg_runs_scored
        wl = sinn_wicks
        rc = fg_runs_scored
        wt = finn_wicks
        bb = sball_bowled
        bf = fball_bowled
        bat_first = False
        bat_second = True
        ht = False
        at = True
    
    team_df = pd.DataFrame({'team_name' : [h_team, a_team], 'match_id' : [match_number, match_number], 
                            'runs_scored' : [rs, rc], 'wickets_lost' : [wl, wt],
                            'runs_conceded' : [rc, rs], 'wickets_taken' : [wt, wl], 'date' : [date, date],
                            'comp' : [gender, gender], 'balls_bowled' : [bb, bf], 'balls_faced' : [bf, bb],
                            'bat_first' : [bat_first, bat_second], 'home' : [ht, at]})
    
    team_info = team_info.append(team_df, ignore_index = True)
    
    p_names = []
    p_teams = []
    p_match_ids = []
    p_gender = []
    p_runs_scored = []
    p_runs_conceded = []
    p_wickets = []
    p_outs = []
    p_catches = []
    p_balls = []
    p_fours = []
    p_sixes = []
    p_extras = []
    p_poms = []
    p_dates = []
    p_balls_bowled = []

    
    for t in runs_scored.keys():
        for p in runs_scored[t].keys():
            if p == pom:
                p_poms.append(True)
            else:
                p_poms.append(False)
            p_names.append(p)
            p_teams.append(t)
            p_gender.append(gender)
            p_match_ids.append(match_number)
            p_runs_scored.append(runs_scored[t][p])
            p_runs_conceded.append(runs_con[t][p])
            p_wickets.append(wickets[t][p])
            p_outs.append(outs[t][p])
            p_catches.append(catches[t][p])
            p_balls.append(balls[t][p])
            p_fours.append(fours[t][p])
            p_sixes.append(sixes[t][p])
            p_extras.append(extras[t][p])
            p_dates.append(date)
            p_balls_bowled.append(balls_bowled[t][p])
            
    player_df = pd.DataFrame({'player_name' : p_names, 'player_team' : p_teams, 'match_id' : p_match_ids,
                 'player_gender' : p_gender, 'runs_scored' : p_runs_scored, 'runs_conceded' : p_runs_conceded,
                 'wickets_taken' : p_wickets, 'out' : p_outs, 'catches' : p_catches, 'balls_faced' : p_balls,
                 'fours' : p_fours, 'sixes' : p_sixes, 'extras' : p_extras, 'POM' : p_poms, 'date' : p_dates,
                 'balls_bowled' : p_balls_bowled})
    player_info = player_info.append(player_df, ignore_index = True)
    match_number += 1
match_df =pd.DataFrame({'match_id' : match_ids, 'home_team' : h_teams, 'away_team' : a_teams,
                        'city' : cities, 'venue' : venues, 'date' : dates, 'competition' : genders,
                        'toss_winner' : toss_winners, 'toss_decision' : toss_decisions, 'bat_first' : bat_firsts, 
                        'first_innings_runs' : first_inn_runs, 'first_innings_wickets' : first_inn_wicks,
                        'second_innings_runs' : second_inn_runs, 'second_innings_wickets' : second_inn_wicks,
                        'winner' : winners, 'by' : outcomes, 'first_innings_balls' : fballs,
                        'second_innings_balls' : sballs})


C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10100\4082732160.py:338: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  team_info = team_info.append(team_df, ignore_index = True)
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10100\4082732160.py:385: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  player_info = player_info.append(player_df, ignore_index = True)
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10100\4082732160.py:385: FutureWarning: Behavior when concatenating bool-dtype and numeric-dtype arrays is deprecated; in a future version these will cast to object dtype (instead of coercing bools to numeric values). To retain the old behavior, explicitly cast bool-dtype arrays to numeric dtype.
  player_info = player_info.append(player_df, ignore_index = True)
C:\Users\Pu

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10100\4082732160.py:385: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  player_info = player_info.append(player_df, ignore_index = True)
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10100\4082732160.py:338: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  team_info = team_info.append(team_df, ignore_index = True)
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10100\4082732160.py:338: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  team_info = team_info.append(team_df, ignore_index = True)
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10100\4082732160.py:385: FutureWarning: The frame.append 

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10100\4082732160.py:338: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  team_info = team_info.append(team_df, ignore_index = True)
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10100\4082732160.py:338: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  team_info = team_info.append(team_df, ignore_index = True)
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10100\4082732160.py:385: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  player_info = player_info.append(player_df, ignore_index = True)
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10100\4082732160.py:338: FutureWarning: The frame.append 

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10100\4082732160.py:338: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  team_info = team_info.append(team_df, ignore_index = True)
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10100\4082732160.py:338: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  team_info = team_info.append(team_df, ignore_index = True)
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10100\4082732160.py:385: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  player_info = player_info.append(player_df, ignore_index = True)
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10100\4082732160.py:338: FutureWarning: The frame.append 

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10100\4082732160.py:338: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  team_info = team_info.append(team_df, ignore_index = True)
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10100\4082732160.py:338: FutureWarning: In a future version, object-dtype columns with all-bool values will not be included in reductions with bool_only=True. Explicitly cast to bool dtype instead.
  team_info = team_info.append(team_df, ignore_index = True)
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10100\4082732160.py:385: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  player_info = player_info.append(player_df, ignore_index = True)
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10100\4082732160.py:338: FutureWarning: The frame.append 

In [419]:
player_info

,player_name,player_team,match_id,player_gender,runs_scored,runs_conceded,wickets_taken,out,catches,balls_faced,fours,sixes,extras,POM,date,balls_bowled
0,JJ Roy,Oval Invincibles,1.0,male,20.0,0.0,0.0,1.0,2.0,12.0,4.0,0.0,0.0,0.0,2021-07-22,0.0
1,SM Curran,Oval Invincibles,1.0,male,6.0,28.0,2.0,1.0,1.0,3.0,0.0,1.0,2.0,0.0,2021-07-22,20.0
2,SP Narine,Oval Invincibles,1.0,male,2.0,23.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0,2021-07-22,20.0
3,WG Jacks,Oval Invincibles,1.0,male,15.0,8.0,0.0,1.0,0.0,9.0,1.0,1.0,0.0,0.0,2021-07-22,5.0
4,SW Billings,Oval Invincibles,1.0,male,49.0,0.0,0.0,1.0,1.0,30.0,4.0,2.0,0.0,1.0,2021-07-22,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3911,TK Curran,Oval Invincibles,178.0,male,18.0,31.0,2.0,0.0,0.0,10.0,3.0,0.0,2.0,0.0,2023-08-21,20.0
3912,NA Sowter,Oval Invincibles,178.0,male,0.0,18.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-08-21,10.0
3913,A Zampa,Oval Invincibles,178.0,male,0.0,21.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2023-08-21,20.0
3914,SH Johnson,Oval Invincibles,178.0,male,0.0,16.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2023-08-21,10.0


In [420]:
team_df

,team_name,match_id,runs_scored,wickets_lost,runs_conceded,wickets_taken,date,comp,balls_bowled,balls_faced,bat_first,home
0,Oval Invincibles,178,151,5,148,7,2023-08-21,male,92,100,False,False
1,Trent Rockets,178,148,7,151,5,2023-08-21,male,100,92,True,True


In [421]:
team_info['date'] = pd.to_datetime(team_info.date, format = '%Y-%m-%d')

team_names = []
genders = []
match_ids = []
curr_1st_bat_avs = []
curr_2nd_bat_avs = []
curr_bat_per_wick = []
curr_1st_bowl_avs = []
curr_2nd_bowl_avs = []
curr_wick_avs = []
curr_wick_loss = []
curr_run_per_wick = []
curr_runs_per_ball = []
curr_con_per_ball = []

for t in team_info['team_name'].unique():
    game = 1
    for g in team_info['comp'].unique():
        subset = team_info[(team_info.team_name == t) & (team_info.comp == g)].dropna().reset_index(drop = True)
        bat_1st = 1
        bat_2nd = 1
        first_runs_scored = 0
        first_runs_conceded = 0
        second_runs_scored = 0
        second_runs_conceded = 0
        wickets_taken = 0
        wickets_lost = 0
        total_runs_scored = 0
        total_runs_conceded = 0
        balls_bowled = 0
        balls_faced = 0
        for i in range(len(subset)):
            match_ids.append(subset.loc[i, 'match_id'])
            
            runs = int(subset.loc[i, 'runs_scored'])
            total_runs_scored += runs
            if subset.loc[i, 'bat_first'] == True:
                first_runs_scored += runs
            else:
                second_runs_scored += runs
            
            wicks = subset.loc[i, 'wickets_lost']
            wickets_lost += wicks
            runs_con = subset.loc[i, 'runs_conceded']
            total_runs_conceded += runs_con
            if subset.loc[i, 'bat_first'] == True:
                first_runs_conceded += runs_con
            else:
                second_runs_conceded += runs_con
            wick_take = subset.loc[i, 'wickets_taken']
            wickets_taken += wick_take
            
            balls_bowled += subset.loc[i, 'balls_bowled']
            balls_faced += subset.loc[i, 'balls_faced']
            
            team_names.append(t)
            genders.append(g)
            curr_1st_bat_avs.append(round(first_runs_scored/bat_1st, 1))
            curr_2nd_bat_avs.append(round(second_runs_scored/bat_2nd, 1))
            curr_bat_per_wick.append(round(total_runs_scored/wickets_lost, 1))
            curr_1st_bowl_avs.append(round(first_runs_conceded/bat_1st, 1))
            curr_2nd_bowl_avs.append(round(second_runs_conceded/bat_2nd, 1))
            curr_run_per_wick.append(round(total_runs_conceded/wickets_taken, 1))
            curr_wick_avs.append(round(wickets_taken/game, 1))
            curr_wick_loss.append(round(wickets_lost/game, 1))
            curr_runs_per_ball.append(round(total_runs_scored/balls_faced, 2))
            curr_con_per_ball.append(round(total_runs_conceded/balls_bowled, 2))
            game+=1
            if subset.loc[i, 'bat_first'] == True:
                bat_1st += 1
            else:
                bat_2nd += 1

In [422]:
team_stats_per_match = pd.DataFrame({'match_id' : match_ids, 'team' : team_names, 'competition' : genders,
                                     'average_first_innings_runs_scored_per_game' : curr_1st_bat_avs,
                                     'average_second_innings_runs_scored_per_game' : curr_2nd_bat_avs,
                                     'average_wickets_lost_per_game' : curr_wick_loss,
                                     'average_runs_per_wicket_lost' : curr_bat_per_wick,
                                     'average_first_innings_runs_conceded_per_game' : curr_1st_bowl_avs,
                                     'average_second_innings_runs_conceded_per_game' : curr_2nd_bowl_avs,
                                     'average_wickets_taken_per_game' : curr_wick_avs,
                                     'average_runs_per_wicket_taken' : curr_run_per_wick,
                                     'average_runs_scored_per_ball' : curr_runs_per_ball,
                                     'average_runs_conceded_per_ball' : curr_con_per_ball})

In [423]:
team_stats_per_match

,match_id,team,competition,average_first_innings_runs_scored_per_game,average_second_innings_runs_scored_per_game,average_wickets_lost_per_game,average_runs_per_wicket_lost,average_first_innings_runs_conceded_per_game,average_second_innings_runs_conceded_per_game,average_wickets_taken_per_game,average_runs_per_wicket_taken,average_runs_scored_per_ball,average_runs_conceded_per_ball
0,1.0,Manchester Originals,male,0.0,136.0,7.0,19.4,0.0,145.0,8.0,18.1,1.36,1.45
1,5.0,Manchester Originals,male,0.0,112.5,5.5,20.5,0.0,116.0,9.0,12.9,1.22,1.34
2,10.0,Manchester Originals,male,0.0,126.0,4.7,27.0,0.0,127.3,8.0,15.9,1.33,1.43
3,22.0,Manchester Originals,male,0.0,128.5,5.8,22.3,0.0,131.0,7.5,17.5,1.34,1.42
4,24.0,Manchester Originals,male,0.0,129.0,6.6,19.5,0.0,144.8,7.0,20.7,1.33,1.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...
343,134.0,Welsh Fire,female,120.0,103.9,2.8,18.3,118.5,115.0,1.9,27.8,1.31,1.26
344,137.0,Welsh Fire,female,122.0,103.9,2.9,18.7,120.8,115.0,2.0,26.8,1.32,1.27
345,140.0,Welsh Fire,female,126.5,103.9,2.9,19.8,122.2,115.0,2.1,26.9,1.35,1.28
346,148.0,Welsh Fire,female,129.0,103.9,2.9,20.4,123.4,115.0,2.2,26.4,1.36,1.29


In [176]:
player_info[player_info.balls_faced > 0]

,player_name,player_team,match_id,player_gender,runs_scored,runs_conceded,wickets_taken,out,catches,ball_faced,fours,sixes,extras,POM,date
0,JJ Roy,Oval Invincibles,2021_1_male,male,20.0,0.0,0.0,1.0,2.0,12.0,4.0,0.0,0.0,0.0,2021-07-22
1,SM Curran,Oval Invincibles,2021_1_male,male,6.0,28.0,2.0,1.0,1.0,3.0,0.0,1.0,2.0,0.0,2021-07-22
2,SP Narine,Oval Invincibles,2021_1_male,male,2.0,23.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0,2021-07-22
3,WG Jacks,Oval Invincibles,2021_1_male,male,15.0,8.0,0.0,1.0,0.0,9.0,1.0,1.0,0.0,0.0,2021-07-22
4,SW Billings,Oval Invincibles,2021_1_male,male,49.0,0.0,0.0,1.0,1.0,30.0,4.0,2.0,0.0,1.0,2021-07-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3911,TK Curran,Oval Invincibles,2023_178_male,male,18.0,31.0,2.0,0.0,0.0,10.0,3.0,0.0,2.0,0.0,2023-08-21
3912,NA Sowter,Oval Invincibles,2023_178_male,male,0.0,18.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-08-21
3913,A Zampa,Oval Invincibles,2023_178_male,male,0.0,21.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2023-08-21
3914,SH Johnson,Oval Invincibles,2023_178_male,male,0.0,16.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2023-08-21


In [424]:
player_info['date'] = pd.to_datetime(player_info.date, format = '%Y-%m-%d')

names = []
curr_team = []
gender = []
games = []
runs_scored = []
not_outs = []
balls_faced = []
runs_conceded = []
wickets_taken = []
extras_conceded = []
catches = []
fours = []
sixes = []
times_batted = []
poms = []
balls_bowled = []
high_score = []
best_bowling = []


for p in player_info.player_name.unique():
    
    subset = player_info[player_info.player_name == p].reset_index(drop = True)
    
    names.append(p)
    curr_team.append(subset.loc[len(subset) - 1, 'player_team'])
    games.append(len(subset))
    runs_scored.append(int(subset.runs_scored.sum()))
    tb = len(subset[subset.balls_faced>0])

    no = int(len(subset[subset.balls_faced>0]) - subset.out.sum())
    if no < 1:
        no = np.nan
    not_outs.append(no)
    balls_faced.append(int(subset.balls_faced.sum()))
    fours.append(int(subset.fours.sum()))
    sixes.append(int(subset.sixes.sum()))
    hs = int(subset.runs_scored.max())
    if tb < 1:
        hs = np.nan
    high_score.append(hs)
    balls_bowled.append(int(subset.balls_bowled.sum()))
    runs_conceded.append(int(subset.runs_conceded.sum()))
    extras_conceded.append(int(subset.extras.sum()))
    cat = subset.catches.sum()
    gender.append(subset.loc[0, 'player_gender'])
    times_batted.append(tb)
    if p in sub_catches.keys():
        cat += sub_catches[p]
    catches.append(int(cat))   
    poms.append(int(subset.POM.sum()))
    max_wick = int(subset.sort_values(by = 'wickets_taken', ascending = False).wickets_taken.max())

    accomp_run = int(subset[subset.wickets_taken == max_wick].sort_values(by = 'runs_conceded').runs_conceded.min())
    b_bowl = str(accomp_run) +'-' + str(max_wick)
    if b_bowl == '0-0':
        b_bowl = np.nan
    best_bowling.append(b_bowl)
    
    
player_stats = pd.DataFrame({'player_name' : names, 'gender' : gender, 'current_team' : curr_team, 'games_played' : games,
                             'runs_scored' : runs_scored, 'innings' : times_batted, 
                             'not_outs' : not_outs, 'balls_faced' : balls_faced,
                             'fours' : fours, 'sixes' :sixes, 'high_score' : high_score,
                             'balls_bowled' : balls_bowled, 'runs_conceded' : runs_conceded, 
                             'best_bowling' : best_bowling,
                             'extras' : extras_conceded, 'catches' : catches, 'POMs' :poms})

In [425]:
player_stats

,player_name,gender,current_team,games_played,runs_scored,innings,not_outs,balls_faced,fours,sixes,high_score,balls_bowled,runs_conceded,best_bowling,extras,catches,POMs
0,JJ Roy,male,Oval Invincibles,21,391,21,2.0,293,42,13,59.0,0,0,NaN,0,9,1
1,SM Curran,male,Oval Invincibles,17,318,17,1.0,245,20,16,60.0,321,502,16-3,24,10,2
2,SP Narine,male,Oval Invincibles,18,145,14,4.0,107,8,11,23.0,345,413,12-3,10,2,1
3,WG Jacks,male,Oval Invincibles,22,620,22,1.0,368,60,34,108.0,60,73,8-2,5,9,3
4,SW Billings,male,Oval Invincibles,22,349,22,6.0,244,28,14,76.0,0,0,NaN,0,10,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,CR Woakes,male,Birmingham Phoenix,3,20,3,1.0,20,0,1,16.0,55,71,21-1,4,0,0
359,JA Turner,male,Trent Rockets,1,0,0,NaN,0,0,0,NaN,20,23,23-3,2,0,0
360,TS Muyeye,male,Oval Invincibles,3,11,2,NaN,13,1,0,11.0,0,0,NaN,0,2,0
361,Zaman Khan,male,Manchester Originals,2,0,0,NaN,0,0,0,NaN,35,61,18-0,2,1,0


In [270]:
subset 

,player_name,player_team,match_id,player_gender,runs_scored,runs_conceded,wickets_taken,out,catches,balls_faced,fours,sixes,extras,POM,date,balls_bowled
0,MJ Henry,Welsh Fire,2023_177_male,male,0.0,17.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2023-08-20,20.0


In [271]:
subset[subset.wickets_taken == max_wick].sort_values(by = 'runs_conceded').runs_conceded.max()

17.0

In [272]:
player_stats.sort_values(by = 'high_score', ascending = False)

,player_name,gender,current_team,games_played,runs_scored,innings,not_outs,balls_faced,fours,sixes,high_score,balls_bowled,runs_conceded,best_bowling,extras,catches,POMs
219,TT Beaumont,female,Welsh Fire,19,530,19,1.0,412,78,6,118.0,0,0,NaN,0,8,1
3,WG Jacks,male,Oval Invincibles,22,620,22,1.0,368,60,34,108.0,60,73,8-2,5,9,3
105,WCF Smeed,male,Birmingham Phoenix,20,428,20,2.0,267,42,22,101.0,0,0,NaN,0,11,2
57,DJ Malan,male,Trent Rockets,22,619,21,2.0,434,58,24,98.0,0,0,NaN,0,4,4
244,BL Mooney,female,London Spirit,6,205,6,2.0,132,32,1,97.0,0,0,NaN,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
335,C Brewer,female,Birmingham Phoenix,1,0,0,NaN,0,0,0,NaN,10,26,26-0,1,0,0
346,JC Tongue,male,Manchester Originals,4,0,0,NaN,0,0,0,NaN,66,107,32-3,6,1,0
359,JA Turner,male,Trent Rockets,1,0,0,NaN,0,0,0,NaN,20,23,23-3,2,0,0
361,Zaman Khan,male,Manchester Originals,2,0,0,NaN,0,0,0,NaN,35,61,18-0,2,1,0


In [426]:
match_info = match_df[['match_id', 'date', 'home_team', 'away_team', 'city', 'venue', 'competition',
                       'toss_winner', 'toss_decision', 'bat_first', 'winner', 'by']]

In [402]:
ven_dic = {}

for v in match_info.venue.unique():
    print(v)
    inp = input('team')
    ven_dic[inp] = v

Kennington Oval, London
teamOval Invincibles
Edgbaston, Birmingham
teamBirmingham Phoenix
Trent Bridge, Nottingham
teamTrent Rockets
Headingley, Leeds
teamNorthern Superchargers
Old Trafford, Manchester
teamManchester Originals
Sophia Gardens, Cardiff
teamWelsh Fire
Lord's, London
teamLondon Spirit
The Rose Bowl, Southampton
teamSouthern Brave


In [342]:
todec = match_info.groupby(['competition', 'toss_winner', 'toss_decision']).count().loc[g].loc[t]

-2

In [427]:
names = []
genders = []
home_ven = []
wins = []
losses = []
runs = []
cons = []
wickets =[]
nr = []
toss_wins = []
toss_decs = []
games = []

for t in team_info.team_name.unique():
    for g in team_info.comp.unique():
        subset = team_info[(team_info.team_name == t) & (team_info.comp == g)].reset_index(drop = True)
        names.append(t)
        genders.append(g)
        home_ven.append(ven_dic[t])
        runs.append(int(subset.runs_scored.sum()))
        cons.append(int(subset.runs_conceded.sum()))
        games.append(len(subset))
        wickets.append(int(subset.wickets_lost.sum()))
        wins.append(match_info.groupby(['competition', 'winner']).count().loc[g].loc[t].loc['by'])
        toss_wins.append(match_info.groupby(['competition', 'toss_winner']).count().loc[g].loc[t].loc['by'])
        try:
            hna = match_info[(match_info.by == 'no result') & (match_info.competition == g)].home_team.value_counts()[t]
        except:
            hna = 0
        try:
            ana = match_info[(match_info.by == 'no result') & (match_info.competition == g)].away_team.value_counts()[t]
        except:
            ana = 0
        todec = match_info.groupby(['competition', 'toss_winner', 'toss_decision']).count().loc[g].loc[t]
        if todec.loc['bat','by'] > todec.loc['field', 'by']:
            toss_decs.append('bat')
        else:
            toss_decs.append('field')
        nores = hna + ana
        nr.append(nores)
        losses.append(len(subset) - nores - match_info.groupby(['competition', 'winner']).count().loc[g].loc[t].loc['by'])

In [428]:
team_stats_overall = pd.DataFrame({'team_name' : names, 'team' : genders, 'home_venue' : home_ven, 'games_played' : games,
                                  'wins' : wins, 'losses' : losses, 'no_results' : nr, 'runs_scored' : runs,
                                   'runs_conceded' : cons, 'wickets_taken' :wickets, 'toss_wins' : toss_wins,
                                  'most_common_toss_decision' : toss_decs})

In [357]:
match_df.groupby('venue').mean()

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10100\3782190418.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  match_df.groupby('venue').mean()


,first_innings_runs,first_innings_wickets,second_innings_runs,second_innings_wickets,first_innings_balls,second_innings_balls
venue,,,,,,
"Edgbaston, Birmingham",145.210526,6.157895,128.526316,6.105263,100.052632,90.947368
"Headingley, Leeds",145.727273,5.636364,137.047619,5.714286,97.954545,93.619048
"Kennington Oval, London",136.333333,6.875000,129.166667,6.416667,98.250000,92.125000
"Lord's, London",127.040000,6.360000,120.875000,6.166667,96.000000,94.625000
"Old Trafford, Manchester",134.777778,6.166667,118.250000,6.125000,96.222222,92.250000
"Sophia Gardens, Cardiff",132.227273,6.272727,125.090909,4.772727,97.318182,89.500000
"The Rose Bowl, Southampton",137.200000,5.320000,129.600000,5.080000,98.920000,91.120000
"Trent Bridge, Nottingham",139.565217,6.434783,130.217391,5.434783,99.695652,94.434783


In [429]:
names = []
city = []
games = []
av_score = []

com_dec = []
com_win = []
nrs = []

for v in match_df.venue.unique():
    split = v.split(', ')
    names.append(split[0])
    city.append(split[1])
    subset = match_df[match_df.venue == v].reset_index(drop = True)
    games.append(len(subset))
    nrs.append(len(subset[subset.by == 'no result']))
    av_score.append(round(subset.first_innings_runs.mean(),2))
    com_dec.append(subset.toss_decision.value_counts().index[0])
    bat_win = 0
    for i in range(len(subset)):
        bat_first = subset.loc[i, 'bat_first']
        winner = subset.loc[i, 'winner']

        if bat_first == winner:
            bat_win += 0
                
    if bat_win > len(subset)/2:
        com_win.append('bat')
    else:
        com_win.append('field')

In [430]:
venue_df = pd.DataFrame({'venue_name' : names, 'city' : city, 'games' : games, 'no_results': nrs,
                        'average_first_innings_score' : av_score,
                         'most_common_toss_decision' : com_dec,
                         'most_common_winner' : com_win})

In [431]:
match_info['venue'] = match_info.venue.apply(lambda x: x.split(', ')[0])

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10100\2280350099.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  match_info['venue'] = match_info.venue.apply(lambda x: x.split(', ')[0])


In [432]:
match_info

,match_id,date,home_team,away_team,city,venue,competition,toss_winner,toss_decision,bat_first,winner,by
0,1,2021-07-22,Manchester Originals,Oval Invincibles,London,Kennington Oval,male,Manchester Originals,field,Oval Invincibles,Oval Invincibles,9 runs
1,2,2021-07-23,Birmingham Phoenix,London Spirit,Birmingham,Edgbaston,male,Birmingham Phoenix,field,London Spirit,Birmingham Phoenix,3 wickets
2,3,2021-07-24,Trent Rockets,Southern Brave,Nottingham,Trent Bridge,male,Southern Brave,bat,Southern Brave,Trent Rockets,9 wickets
3,4,2021-07-24,Northern Superchargers,Welsh Fire,Leeds,Headingley,male,Northern Superchargers,field,Welsh Fire,Welsh Fire,5 runs
4,5,2021-07-25,Manchester Originals,Birmingham Phoenix,Manchester,Old Trafford,male,Birmingham Phoenix,bat,Birmingham Phoenix,Manchester Originals,6 wickets
...,...,...,...,...,...,...,...,...,...,...,...,...
173,174,2023-08-19,Birmingham Phoenix,Trent Rockets,Nottingham,Trent Bridge,male,Birmingham Phoenix,field,Trent Rockets,Trent Rockets,46 runs
174,175,2023-08-19,Southern Brave,Oval Invincibles,Southampton,The Rose Bowl,male,Oval Invincibles,bat,Oval Invincibles,Oval Invincibles,8 runs
175,176,2023-08-20,Northern Superchargers,Manchester Originals,Manchester,Old Trafford,male,Manchester Originals,bat,Manchester Originals,Manchester Originals,81 runs
176,177,2023-08-20,Welsh Fire,London Spirit,Cardiff,Sophia Gardens,male,London Spirit,bat,London Spirit,Welsh Fire,6 wickets


In [433]:
player_info.head()

,player_name,player_team,match_id,player_gender,runs_scored,runs_conceded,wickets_taken,out,catches,balls_faced,fours,sixes,extras,POM,date,balls_bowled
0,JJ Roy,Oval Invincibles,1.0,male,20.0,0.0,0.0,1.0,2.0,12.0,4.0,0.0,0.0,0.0,2021-07-22,0.0
1,SM Curran,Oval Invincibles,1.0,male,6.0,28.0,2.0,1.0,1.0,3.0,0.0,1.0,2.0,0.0,2021-07-22,20.0
2,SP Narine,Oval Invincibles,1.0,male,2.0,23.0,1.0,1.0,0.0,5.0,0.0,0.0,2.0,0.0,2021-07-22,20.0
3,WG Jacks,Oval Invincibles,1.0,male,15.0,8.0,0.0,1.0,0.0,9.0,1.0,1.0,0.0,0.0,2021-07-22,5.0
4,SW Billings,Oval Invincibles,1.0,male,49.0,0.0,0.0,1.0,1.0,30.0,4.0,2.0,0.0,1.0,2021-07-22,0.0


In [441]:
names = []
match_ids = []
bat_avs = []
bowl_avs = []
bat_econ = []
bowl_econ = []


for p in player_info.player_name.unique():
    subset = player_info[player_info.player_name == p].reset_index(drop = True)
    
    rs = 0
    rc = 0
    wicks = 0
    outs = 0
    face_balls = 0
    bowl_balls = 0
    
    for i in range(len(subset)):
        rs += subset.loc[i, 'runs_scored']
        rc += subset.loc[i, 'runs_conceded']
        wicks += subset.loc[i, 'wickets_taken']
        outs += subset.loc[i, 'out']
        face_balls += subset.loc[i, 'balls_faced']
        bowl_balls += subset.loc[i, 'balls_bowled']
        names.append(p)
        match_ids.append(subset.loc[i, 'match_id'])
        try:
            bat_avs.append(round(rs/outs, 2))
        except:
            bat_avs.append(np.nan)
        try:
            bowl_avs.append(round(rc/wicks, 2))
        except:
            bowl_avs.append(np.nan)
        try:
            bat_econ.append(round(rs/face_balls,2))
        except:
            bat_econ.append(np.nan)
        try:
            bowl_econ.append(round(rc/bowl_balls,2))
        except:
            bowl_econ.append(np.nan)
        

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10100\87769477.py:33: RuntimeWarning: invalid value encountered in scalar divide
  bowl_avs.append(round(rc/wicks, 2))
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10100\87769477.py:41: RuntimeWarning: invalid value encountered in scalar divide
  bowl_econ.append(round(rc/bowl_balls,2))
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10100\87769477.py:33: RuntimeWarning: divide by zero encountered in scalar divide
  bowl_avs.append(round(rc/wicks, 2))
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10100\87769477.py:29: RuntimeWarning: divide by zero encountered in scalar divide
  bat_avs.append(round(rs/outs, 2))
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10100\87769477.py:29: RuntimeWarning: invalid value encountered in scalar divide
  bat_avs.append(round(rs/outs, 2))
C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_10100\87769477.py:37: RuntimeWarning: inva

In [442]:
player_stats_after_each_game = pd.DataFrame({'player_name' : names, 'match_id' : match_ids,
                                            'batting_average' : bat_avs, 'strike_rate' : bat_econ,
                                             'bowling_average' : bowl_avs, 'runs_per_ball' : bowl_econ})

In [445]:
player_stats_overall.to_csv('overall_player_stats.csv', index = False)  #note, includes catches as substitute fielder
player_performance_per_game.to_csv('player_performance_per_game.csv', index = False)
team_performance_per_game.to_csv('team_performance_per_game.csv', index = False) 
team_stats_after_each_game.to_csv('team_stats_after_each_game.csv', index = False) 
match_info.to_csv('match_information.csv', index = False)
player_stats_after_each_game.to_csv('player_stats_after_each_game.csv', index = False)
team_stats_overall.to_csv('overall_team_stats.csv', index = False)
venue_df.to_csv('venue_stats.csv', index = False)